# Bootstrapping the $\chi^2$ $1\sigma$ confidence level

In [ ]:
import os
import shutil
import numpy as np
import astropy
import dynamite as dyn

In [ ]:
config_file_name = 'user_test_config_ml.yaml'
output_dir = 'bootstrap'
n_scen = 5  # number of kinematic data perturbation scenarios

In [ ]:
if output_dir[-1] != '/':
    output_dir += '/'

In [ ]:
c = dyn.config_reader.Configuration(filename=config_file_name, reset_logging=True, user_logfile='bootstrap', reset_existing_output=False)

In [ ]:
# Some config file checks. It should have n_max_iter = 0 and reattempt_failures = True so that it only runs weight solving on the best model and nothing else.
n_max_iter = c.settings.parameter_space_settings['stopping_criteria']['n_max_iter']
if n_max_iter > 0:
    print(f'***** parameter_space_settings -> stopping_criteria -> n_max_iter SHOULD BE ZERO, but it is {n_max_iter}! *****')
if not c.settings.weight_solver_settings['reattempt_failures']:
    print(f"***** weight_solver_settings -> reattempt_failures SHOULD BE TRUE, but it isn't! *****")

In [ ]:
which_chi2 = c.settings.parameter_space_settings['which_chi2']
model_idx = c.all_models.get_best_n_models_idx(n=1)[0]
model = c.all_models.get_model_from_row(model_idx)
best_chi2 = c.all_models.table[model_idx][which_chi2]
print(f'Selecting minimum {which_chi2} model:\n'
      f'model index = {model_idx}\n{which_chi2} = {best_chi2}\n'
      f'model directory = {model.directory}\n'
      f'orblib directory = {model.directory_noml}')

In [ ]:
def write_pythonscript(directory=''):
    with open(directory + 'run.py', 'w') as f:
        f.write('import dynamite as dyn\n'
                f'c = dyn.config_reader.Configuration("{config_file_name}", reset_logging=True)\n'
                '_ = dyn.model_iterator.ModelIterator(c)\n')

In [ ]:
def perturb_kins(directory, files):
    print(f'Perturbing kinematics in directory {directory}, files: {files}...')

In [ ]:
# create bootstrapping directories: base scenario bs_0000 + n_scen perturbed scenarios
for scen in range(n_scen + 1):  # scen=0 is the base case
    scen_dir = output_dir + f'bs_{scen:04d}/'
    # copy input directory
    shutil.copytree(c.settings.io_settings['input_directory'],
                    scen_dir + c.settings.io_settings['input_directory'],
                    dirs_exist_ok=True)  # creates intermediate directories
    # copy config file
    shutil.copy2(config_file_name, scen_dir)
    # make model directory
    os.makedirs(scen_dir + model.directory, exist_ok=True)
    # copy or link orbit library
    datfil = scen_dir + model.directory_noml + 'datfil'
    if scen == 0:
        # copy orbit library
        shutil.copytree(model.directory_noml + 'datfil', datfil, dirs_exist_ok=True)
    else:  # create a symbolic link for the orbit library to save disk space
        if os.path.isfile(datfil) or os.path.isdir(datfil):
            os.unlink(datfil)
        os.symlink('../../../../bs_0000/' + model.directory_noml + 'datfil', 
                   datfil, 
                   target_is_directory=True)
    # copy all_models table
    all_models_file = c.settings.io_settings['output_directory'] + c.settings.io_settings['all_models_file']
    shutil.copy2(all_models_file, scen_dir + all_models_file)
    # cannot delete all entries from all_models table except model_idx because of get_ml_of_original_orblib()!
    # just set weights_done = all_done = False for the best model
    all_models = astropy.io.ascii.read(scen_dir + all_models_file)  # read all_models table
    all_models[model_idx]['weights_done'] = all_models[model_idx]['all_done'] = False
    # save the all_models table
    all_models.write(scen_dir + all_models_file, format='ascii.ecsv', overwrite=True)
    if scen > 0:
        if c.system.is_bar_disk_system():
            stars = c.system.get_unique_bar_component()
        else:
            stars = c.system.get_unique_triaxial_visible_component()
        kin_files = [kin.datafile for kin in stars.kinematic_data]
        perturb_kins(directory=scen_dir + c.settings.io_settings['input_directory'], files=kin_files)
    # create Python script
    write_pythonscript(scen_dir)